In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# read 데이터
df = pd.read_csv('AFSNT.csv', encoding="cp949")
weather=pd.read_csv('data/2017.csv',encoding="cp949")
df.head()


,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,REG,AOD,IRR,STT,ATT,DLY,DRR,CNL,CNR
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,N,10:05,10:32,N,NaN,N,NaN
1,2017,1,1,일,ARP3,ARP6,J,J1954,SEw3NzE4,A,N,9:30,9:31,N,NaN,N,NaN
2,2017,1,1,일,ARP3,ARP6,J,J1956,SEw3NzE4,A,N,12:45,13:03,N,NaN,N,NaN
3,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,N,13:25,14:09,Y,C02,N,NaN
4,2017,1,1,일,ARP3,ARP6,J,J1958,SEw3NzE4,A,N,16:10,16:31,N,NaN,N,NaN


In [2]:
df=df[df["CNL"]!="Y"]
df.drop(columns=['CNL', 'CNR'], axis=1, inplace=True)
df = df[pd.notnull(df['ATT'])]
df.isnull().sum()

SDT_YY         0
SDT_MM         0
SDT_DD         0
SDT_DY         0
ARP            0
ODP            0
FLO            0
FLT            0
REG            4
AOD            0
IRR            0
STT            0
ATT            0
DLY            0
DRR       860513
dtype: int64

In [3]:
df.rename(columns={'SDT_YY':'Year', 'SDT_MM':'Month', 'SDT_DD':'DAY'}, inplace=True)
df['DAY'] = pd.to_datetime(df[['Year', 'Month', 'DAY']])
df.drop(columns=['Year', 'Month'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 979450 entries, 0 to 987708
Data columns (total 13 columns):
DAY       979450 non-null datetime64[ns]
SDT_DY    979450 non-null object
ARP       979450 non-null object
ODP       979450 non-null object
FLO       979450 non-null object
FLT       979450 non-null object
REG       979446 non-null object
AOD       979450 non-null object
IRR       979450 non-null object
STT       979450 non-null object
ATT       979450 non-null object
DLY       979450 non-null object
DRR       118937 non-null object
dtypes: datetime64[ns](1), object(12)
memory usage: 104.6+ MB


In [4]:
one_hot_dy = pd.get_dummies(df['SDT_DY'])
df = df.drop(['SDT_DY'],axis = 1)
df = df.join(one_hot_dy)


In [5]:
def readWeather(year):
    filename="data/"+str(year)+".csv"
    weather=pd.read_csv(filename,encoding="cp949")
    
    #일시를 데이트타입과 시간을 따로 저장
    weather["일시"] = weather["일시"].astype('str')
    date=weather["일시"].str.split(expand=True)
    weather["DAY"]=pd.to_datetime(date[0])
    weather["hour"]=date[1]
    
    #일시 drop(변경 전 데이터)
    weather.drop(columns=['일시'], axis=1, inplace=True)
    weather = weather.drop(['지면온도(°C)',"지면온도 QC플래그","5cm 지중온도(°C)","10cm 지중온도(°C)","20cm 지중온도(°C)",
                        "30cm 지중온도(°C)","일조(hr)","일조 QC플래그","일사(MJ/m2)"],axis = 1)
    weather.rename(columns={"지점" : "area", "기온(°C)":"temp" ,"기온 QC플래그":"tempFlag",
                        "강수량(mm)":"rain", "강수량 QC플래그":"rainFlag",
                        "풍속(m/s)":"windSpeed","풍속 QC플래그":"windSpeedFlag","풍향(16방위)":"wind","풍향 QC플래그":"windFlag",
                        "습도(%)":"hum","습도 QC플래그":"humFlag","증기압(hPa)":"Vapor","이슬점온도(°C)":"dew","현지기압(hPa)":"hpa","현지기압 QC플래그":"hpaFlag",
                        "해면기압(hPa)":"seeHpa","해면기압 QC플래그":"seeHpaFlag","적설(cm)":"snow","3시간신적설(cm)":"snow3",
                        "시정(10m)":"visible","전운량(10분위)":"cloudTotal","중하층운량(10분위)":"cloudMiddle","운형(운형약어)":"cloudShape",
                        "최저운고(100m )":"coludHeight","지면상태(지면상태코드)":"stateCode","현상번호(국내식)":"weatherCode",
                    
                  }, inplace=True)
    weather["hour"]=weather["hour"].astype("str")
    weather["hour"]=weather["hour"].str.split(":",expand=True)[0]
    weather["hour"].fillna(0)
    weather["hour"]=weather["hour"].astype("int")
    return weather

In [6]:
df.rename(columns={"일":"Sun","월":"Mon","화":"Tue","수":"Wed","목":"Thu","금":"Fri","토":"SAT","일":"Sun"                    
                  }, inplace=True)

In [7]:
df["STT"]=df["STT"].astype("str")
df["hour"]=df["STT"].str.split(":",expand=True)[0]
df["hour"]=df["hour"].astype("int")


In [12]:
df_all=pd.DataFrame()
for i in range(3):
    df_areaD=pd.DataFrame()
    year=2017+i
    weather=readWeather(year)
    df_yearD=df[df["DAY"].dt.year==year]
    for j in range(15):
        arp="ARP"+str(j+1)
        area={"ARP1":108,"ARP2":159,"ARP3":184,"ARP4":143,"ARP5":152,"ARP6":131,"ARP7":165,
              "ARP8":156, "ARP9":168,"ARP10":90,"ARP11":138,"ARP12":192,
              "ARP13":140,"ARP14":114,"ARP15":112}.get(arp)
        df_areaD=df_yearD[df_yearD["ARP"]==arp]
        df_new=pd.DataFrame()
        weatherT=weather[weather["area"]==area]

        df_new=pd.merge(df_areaD,weatherT,on=["DAY","hour"])
        
        if i==0 and j==0:
            df_all=df_new.copy()
        else:
            df_all=df_all.append(df_new)
            
        print(year,area)
        print("df_areaD",len(df_areaD))
        print("weatherT",len(weatherT))
        print("df_new",len(df_new))
        print("df_all", len(df_all),"\n")



2017 108
df_areaD 126503
weatherT 8736
df_new 126186
df_all 126186 

2017 159
df_areaD 49201
weatherT 8736
df_new 49072
df_all 175258 

2017 184
df_areaD 157685
weatherT 8736
df_new 157262
df_all 332520 

2017 143
df_areaD 13526
weatherT 8736
df_new 13490
df_all 346010 

2017 152
df_areaD 5346
weatherT 8736
df_new 5326
df_all 351336 

2017 131
df_areaD 14299
weatherT 8736
df_new 14261
df_all 365597 

2017 165
df_areaD 876
weatherT 8736
df_new 872
df_all 366469 

2017 156
df_areaD 12694
weatherT 8736
df_new 12656
df_all 379125 

2017 168
df_areaD 5050
weatherT 8736
df_new 5036
df_all 384161 

2017 90
df_areaD 1
weatherT 8736
df_new 1
df_all 384162 

2017 138
df_areaD 1372
weatherT 8736
df_new 1368
df_all 385530 

2017 192
df_areaD 1871
weatherT 8736
df_new 1863
df_all 387393 

2017 140
df_areaD 1420
weatherT 8736
df_new 1416
df_all 388809 

2017 114
df_areaD 706
weatherT 8736
df_new 704
df_all 389513 

2017 112
df_areaD 5131
weatherT 8736
df_new 5117
df_all 394630 

2018 108
df_areaD 12

In [9]:
df_all.iloc[0]
print(len(df))
print(len(df_all))

979450
784225


In [ ]:
df_all.to_csv("weatherNot12.csv")